In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import torch.nn.functional as F


In [2]:
# 定义 SparseDenoisingAutoencoder 类
class AE(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(AE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, hidden_size[0]),
            nn.ReLU(),
            nn.Linear(hidden_size[0],hidden_size[1])
        )
        self.decoder = nn.Sequential(
            nn.Linear(hidden_size[1], hidden_size[0]),
            nn.ReLU(),
            nn.Linear(hidden_size[1], input_size),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [3]:
a = [256,128]
b = 400
c = a+[b]
print(c)

[256, 128, 400]


In [4]:
input_size1 = 400
hidden_sizes1 = [256, 128]
input_size2 = 400
hidden_sizes2 =[128,64]

num_epochs = 100
batch_size = 64
learning_rate = 0.001
noise_factor = 0.5

In [5]:
# 加载数据
data = np.load("SpectrumData.npy", allow_pickle=True)
data = data.T
data.shape

(5661, 400)

In [6]:
#归一化
mm = MinMaxScaler()
data = mm.fit_transform(data)
data.shape

(5661, 400)

In [7]:
# 创建数据加载器
data = torch.tensor(data, dtype=torch.float32)
dataset = TensorDataset(data)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# 初始化模型、损失函数和优化器
model1 = AE(input_size1, hidden_sizes1)
model2 = AE(input_size2, hidden_sizes2)
#model = SparseDenoisingAutoencoder(input_size, hidden_sizes)
criterion = nn.MSELoss()
optimizer1 = optim.Adam(model1.parameters(), lr=learning_rate)
optimizer2 = optim.Adam(model1.parameters(), lr=learning_rate)

In [10]:
# 训练 model1
for epoch in range(num_epochs):
    for data in data_loader:
        x = data[0]  # 提取输入张量
        x = x.view(x.size(0), -1)
        
        # 添加噪声
        #noisy_x = x + noise_factor * torch.randn(x.shape)
        #noisy_x = torch.clamp(noisy_x, 0., 1.)

        # 向前传播
        #output1 = model1(noisy_x)
        output1 = model1(x)
        
        # 计算损失
        loss1 = criterion(output1, x)

        # 反向传播和优化
        optimizer1.zero_grad()
        loss1.backward()
        optimizer1.step()

    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch + 1, num_epochs, loss1.item()))

# 获取 model1 的编码器输出
encoded_data = model1.encoder(dataset)
encoded_data = encoded_data.detach()





RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x256 and 128x400)

In [ ]:
# 准备用于 model2 的数据加载器
encoded_loader = torch.utils.data.DataLoader(encoded_data, batch_size=batch_size, shuffle=True)

# 训练 model2
for epoch in range(num_epochs):
    for data in encoded_loader:
        x = data[0]
        x = x.view(x.size(0), -1)
        
        # 添加噪声
        noisy_x = x + noise_factor * torch.randn(x.shape)
        noisy_x = torch.clamp(noisy_x, 0., 1.)

        # 向前传播
        output2 = model2(noisy_x)
        
        # 计算损失
        loss2 = criterion(output2, x)

        # 反向传播和优化
        optimizer2.zero_grad()
        loss2.backward()
        optimizer2.step()

    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch + 1, num_epochs, loss2.item()))